In [12]:
import dask.dataframe as dd
import dask_geopandas
import geopandas as gpd

In [13]:
ddf = dd.read_csv(r"C:\Users\kalid\Downloads\hotosm_eth_buildings_polygons_shp\165_buildings.csv\165_buildings.csv")
ddf.head()

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,12.224620,39.622421,15.3247,0.6525,"POLYGON((39.6224452244214 12.2246140173808, 39...",7G4X6JFC+RXVV
1,9.000375,38.675319,38.1842,0.7778,"POLYGON((38.6753484623462 9.00034834778261, 38...",6GXW2M2G+44WR
2,9.056364,38.996449,43.1938,0.8425,"POLYGON((38.9964808950235 9.05633716689749, 38...",6GXW3X4W+GHWF
3,12.060549,39.300167,79.6959,0.8168,"POLYGON((39.3002212546639 12.0605296971811, 39...",7G4X3862+637V
4,10.990799,37.420991,103.5792,0.8731,"POLYGON((37.4210451613197 10.9907616137558, 37...",7G2VXCRC+899W


In [15]:
gddf = dask_geopandas.from_dask_dataframe(ddf,
                                          geometry=ddf['geometry']
                                          .map_partitions(gpd.GeoSeries.from_wkt,
                                                          meta=gpd.GeoSeries([])))
gddf = gddf.reset_index()
gddf.head()

,index,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,0,12.224620,39.622421,15.3247,0.6525,"POLYGON ((39.62245 12.22461, 39.62243 12.22464...",7G4X6JFC+RXVV
1,1,9.000375,38.675319,38.1842,0.7778,"POLYGON ((38.67535 9.00035, 38.67535 9.00040, ...",6GXW2M2G+44WR
2,2,9.056364,38.996449,43.1938,0.8425,"POLYGON ((38.99648 9.05634, 38.99647 9.05640, ...",6GXW3X4W+GHWF
3,3,12.060549,39.300167,79.6959,0.8168,"POLYGON ((39.30022 12.06053, 39.30019 12.06060...",7G4X3862+637V
4,4,10.990799,37.420991,103.5792,0.8731,"POLYGON ((37.42105 10.99076, 37.42103 10.99085...",7G2VXCRC+899W


In [16]:
gddf.crs = 'EPSG:4326'

In [22]:
aoi = gpd.read_file(r"D:\GIS DATA\GIS & RS data model\GIS\CSA regional new Data\AA\bn14.shp")
aoi.head()

,R_NAME,R_CODE,COUNT,Area,geometry
0,Addis Abeba,14,10,52698.998157,"POLYGON ((481069.948 979151.674, 481077.713 97..."


In [24]:
aoi = aoi.to_crs(gddf.crs)

In [25]:
clipped = dask_geopandas.clip(gddf,aoi)
clipped.head()

,index,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
186237,186237,8.871303,38.753879,116.0393,0.8805,"POLYGON ((38.75392 8.87125, 38.75382 8.87126, ...",6GWWVQC3+GHC2
152571,152571,8.889316,38.746934,18.8384,0.6681,"POLYGON ((38.74691 8.88930, 38.74691 8.88933, ...",6GWWVPQW+PQFQ
2082,2082,8.890605,38.745429,55.6847,0.8764,"POLYGON ((38.74539 8.89064, 38.74544 8.89065, ...",6GWWVPRW+65V8
138165,138165,8.893751,38.747546,46.3691,0.8737,"POLYGON ((38.74758 8.89371, 38.74750 8.89374, ...",6GWWVPVX+G233
227168,227168,8.893822,38.747359,40.0902,0.8324,"POLYGON ((38.74739 8.89379, 38.74732 8.89381, ...",6GWWVPVW+GWHV


In [26]:
clipped = clipped.compute()
clipped.to_file(r'C:\Users\kalid\Downloads\hotosm_eth_buildings_polygons_shp\output\buildings.shp')

C:\Users\kalid\AppData\Local\Temp\ipykernel_6956\1388151312.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  clipped.to_file(r'C:\Users\kalid\Downloads\hotosm_eth_buildings_polygons_shp\output\buildings.shp')
